In [1]:
import os
from pathlib import Path

import xarray as xr
import pandas as pd
import numpy as np

In [2]:
DIR_DATA = Path(os.path.dirname(os.path.abspath(''))).resolve() / "data" / "processed"
FILE_CURRENTS = "OSCAR_L4_OC_FINAL_V2.0"
FILE_SST = "OSTIA-UKMO-L4-GLOB-REP-v2.0"
FILE_GEOMETRY = "ECCO_L4_GEOMETRY_05DEG_V4R4"
print(f"Datasets directory:\n{DIR_DATA}")
print(f"Found OSCAR currents data? {Path(DIR_DATA / FILE_CURRENTS).is_dir()}")
print(f"Found OSTIA SST data? {Path(DIR_DATA / FILE_SST).is_dir()}")
print(f"Found ECCO geometry data? {Path(DIR_DATA / FILE_GEOMETRY).is_dir()}")

Datasets directory:
/home/isekar/Documents/projects/NASA_SpaceApps_2025/Project/data/processed
Found OSCAR currents data? True
Found OSTIA SST data? True
Found ECCO geometry data? True


In [3]:
currents = xr.open_zarr(DIR_DATA / FILE_CURRENTS / "CURRENTS.zarr")
sst = xr.open_zarr(DIR_DATA / FILE_SST / "SST.zarr")
geometry = xr.open_dataset(DIR_DATA / FILE_GEOMETRY / "GRID_GEOMETRY_ECCO_V4r4_latlon_0p50deg.nc")

In [4]:
print(currents)

<xarray.Dataset> Size: 2GB
Dimensions:  (time: 4626, lon: 261, lat: 201)
Coordinates:
  * lat      (lat) float64 2kB 15.0 15.25 15.5 15.75 ... 64.25 64.5 64.75 65.0
  * lon      (lon) float64 2kB -180.0 -179.8 -179.5 ... -115.5 -115.2 -115.0
  * time     (time) object 37kB 2001-01-01 00:00:00 ... 2013-08-31 00:00:00
Data variables:
    u        (time, lon, lat) float32 971MB dask.array<chunksize=(1, 261, 201), meta=np.ndarray>
    v        (time, lon, lat) float32 971MB dask.array<chunksize=(1, 261, 201), meta=np.ndarray>
Attributes: (12/33)
    title:                      Ocean Surface Current Analyses Real-time (OSC...
    summary:                    Global, daily, 0.25 degree geostrophic and to...
    keywords:                   ocean currents,ocean circulation,surface curr...
    Conventions:                CF-1.8 Standard Names v77, ACDD-1.3, netcdf 4...
    id:                         OSCAR_L4_OC_FINAL_V2.0
    history:                    OSCAR 0.25 degree daily version 2.0 repla

In [5]:
'''
W = s_n^2 + s_s^2 - w^2

s_n = du/dx - dv/dy
s_s = dv/dx + du/dy
w = dv/dx - du/dx

Eddy <-- W < -0.2 * std_W
cyclonic <-- w > 0
anticyclonic <-- w < 0
'''

'\nW = s_n^2 + s_s^2 - w^2\n\ns_n = du/dx - dv/dy\ns_s = dv/dx + du/dy\nw = dv/dx - du/dx\n\nEddy <-- W < -0.2 * std_W\ncyclonic <-- w > 0\nanticyclonic <-- w < 0\n'

In [6]:
R = 6371 * 1e3

lat_rad = np.deg2rad(currents['lat'])
lon_rad = np.deg2rad(currents['lon'])

dx = R * np.cos(lat_rad) * np.deg2rad(currents.lon.diff(dim='lon').mean())
dy = R * np.deg2rad(currents.lat.diff(dim='lat').mean())

dx, dy = xr.align(dx.rename({'lat': 'lat'}), dy)

dudx = currents["u"].differentiate(coord='lon') / dx
dudy = currents["u"].differentiate(coord='lat') / dy
dvdx = currents["v"].differentiate(coord='lon') / dx
dvdy = currents["v"].differentiate(coord='lat') / dy

s_n = dudx - dvdy
s_s = dvdx + dudy
w = dvdx - dudy

W = s_n**2 + s_s**2 - w**2
deviation_W = W.std(dim=["time", "lat", "lon"], skipna=True)
threshold_W = -0.2 * deviation_W

currents["vorticity"] = w
currents["Okubo-Weiss"] = W

cyclonic_condition = (currents["Okubo-Weiss"] < threshold_W) & (currents["vorticity"] < 0)
anticyclonic_condition = (currents["Okubo-Weiss"] < threshold_W) & (currents["vorticity"] > 0)

currents["cyclonic"] = xr.where(cyclonic_condition, 1, 0)
currents["anticyclonic"] = xr.where(anticyclonic_condition, 1, 0)

currents[['anticyclonic', 'cyclonic']] = currents[['anticyclonic', 'cyclonic']].astype(np.bool)
currents = currents.drop_vars(['vorticity', 'Okubo-Weiss'])

currents.to_zarr(DIR_DATA / FILE_CURRENTS / "CURRENTS_EDDIES.zarr", mode='w')

/home/isekar/.conda/envs/SpaceApps2025/lib/python3.13/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
